In [1]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('./data.csv')
sorted_df = df.sort_values(by=['amount', 'diff'], ascending=False)

print('\n')
print(sorted_df.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 72590 entries, 8408 to 13683
Data columns (total 17 columns):
cityid                   72590 non-null int64
takeoffdate              72590 non-null object
diff                     72590 non-null int64
cancel_num               72590 non-null int64
flight_num               72590 non-null int64
bkd_acc_c_sum            72590 non-null int64
bkd_acc_c_real_sum       72590 non-null int64
bkd_acc_q_sum            72590 non-null int64
bkd_acc_q_real_sum       72590 non-null int64
income_acc_c_real_avg    72590 non-null float64
all_seats                72590 non-null int64
direction                72590 non-null object
uv                       72590 non-null int64
pv_1                     72590 non-null int64
pv_2                     72590 non-null int64
amount                   72590 non-null float64
avg_loadfactor           72590 non-null float64
dtypes: float64(3), int64(12), object(2)
memory usage: 10.0+ MB
None


In [2]:
sorted_df.head()

,cityid,takeoffdate,diff,cancel_num,flight_num,bkd_acc_c_sum,bkd_acc_c_real_sum,bkd_acc_q_sum,bkd_acc_q_real_sum,income_acc_c_real_avg,all_seats,direction,uv,pv_1,pv_2,amount,avg_loadfactor
8408,1,2020-01-21,60,5,774,6243,6243,2107,2107,8572.851643,197345,出港,3684,5164,5163,135555.0,0.914923
64526,1,2020-01-21,59,5,774,6496,6496,2161,2161,8921.784038,197345,出港,6636,9484,9483,135555.0,0.914923
49600,1,2020-01-21,58,5,774,6721,6721,2246,2246,9195.167136,197345,出港,9706,13809,13808,135555.0,0.914923
57068,1,2020-01-21,57,5,774,6928,6928,2321,2321,9451.802817,197345,出港,13851,19759,19756,135555.0,0.914923
60761,1,2020-01-21,56,5,774,7142,7142,2409,2409,9777.624413,197345,出港,17997,25682,25676,135555.0,0.914923


In [3]:
import datetime
import calendar

dirdict = {
    '进港': 'in',
    '出港': 'out'
}

def is_weekend(daystr):
    daynums = list(map(int, daystr.split('-')))
    date = datetime.date(*daynums)
    return 'yes' if date.weekday() == 5 or date.weekday() == 6 else 'no'

output_df = sorted_df.copy()
output_df['is_weekend'] = sorted_df['takeoffdate'].apply(is_weekend)
output_df['year'] = sorted_df['takeoffdate'].apply(lambda x: float(x.split('-')[0]))
output_df['month'] = sorted_df['takeoffdate'].apply(lambda x: float(x.split('-')[1]))
output_df['day'] = sorted_df['takeoffdate'].apply(lambda x: float(x.split('-')[2]))
output_df['cityid'] = sorted_df['cityid'].apply(lambda x: '_' + str(x))
output_df['direction'] = sorted_df['direction'].apply(lambda x: dirdict[x])

output_df = output_df.sort_values(by=['takeoffdate'])
output_df.to_csv('single_line.csv')
output_df.head()

,cityid,takeoffdate,diff,cancel_num,flight_num,bkd_acc_c_sum,bkd_acc_c_real_sum,bkd_acc_q_sum,bkd_acc_q_real_sum,income_acc_c_real_avg,...,direction,uv,pv_1,pv_2,amount,avg_loadfactor,is_weekend,year,month,day
63660,_3,2020-01-01,56,7,191,336,336,158,158,260.348243,...,in,2350,3221,3221,24772.0,0.807065,no,2020.0,1.0,1.0
11308,_1,2020-01-01,47,15,717,3741,3741,994,994,2430.499508,...,out,30823,41309,41303,98922.0,0.726954,no,2020.0,1.0,1.0
63641,_1,2020-01-01,46,15,717,3820,3820,1023,1023,2497.416912,...,out,33088,44358,44351,98922.0,0.726954,no,2020.0,1.0,1.0
56165,_1,2020-01-01,45,15,717,3916,3916,1059,1059,2576.385447,...,out,35862,48096,48089,98922.0,0.726954,no,2020.0,1.0,1.0
11307,_1,2020-01-01,44,15,717,4014,4014,1105,1105,2647.820059,...,out,39191,52640,52632,98922.0,0.726954,no,2020.0,1.0,1.0


## Data preprocessing

- Divide categorical columns and numerical columns
- Seperate them into two dict
- Categorical columns: list their vocabulary list
- Numerical columns: list their mean and stddev values

In [4]:
import numpy as np
import json

EPSILON = 1e-6

catcols, numcols = {}, {}
COLUMNS = output_df.columns
for col in COLUMNS:
    if output_df[col].dtype == np.int64 or output_df[col].dtype == np.float64:
        col_std = output_df[col].std()
        if col_std < EPSILON:
            col_std = EPSILON
        numcols[col] = [output_df[col].mean(), col_std]
    else:
        catcols[col] = list(set(output_df[col].values.tolist()))
        
with open('info.json', 'w') as fd:
    jsoninfo = {'catcols': catcols, 'numcols': numcols}
    json.dump(jsoninfo, fd, allow_nan=False, indent=2)
print('OK')

OK


In [5]:
import json

# 1. seperate train/eval
# 2. normalize labels
df = pd.read_csv('single_line.csv')
with open('./info.json', 'r') as fd:
    jsoninfo = json.load(fd)
label_names = ['amount', 'avg_loadfactor']

for label in label_names:
    df[label] = df[label].apply(lambda x: (x - jsoninfo['numcols'][label][0]) / jsoninfo['numcols'][label][1])
    
datasize = len(df)
eval_size = datasize // 5
train_size = datasize - eval_size
print('[*] data_size={} train_size={} eval_size={}'.format(datasize, train_size, eval_size))

df_train = df.iloc[: train_size]
df_eval = df.iloc[train_size: ]

df_train.to_csv('./single_line_train.csv')
df_eval.to_csv('./single_line_test.csv')

df_train.describe()

[*] data_size=72590 train_size=58072 eval_size=14518


,Unnamed: 0,diff,cancel_num,flight_num,bkd_acc_c_sum,bkd_acc_c_real_sum,bkd_acc_q_sum,bkd_acc_q_real_sum,income_acc_c_real_avg,all_seats,uv,pv_1,pv_2,amount,avg_loadfactor,year,month,day
count,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,58072.000000,5.807200e+04,5.807200e+04,5.807200e+04,58072.000000,58072.000000,58072.0,58072.000000,58072.000000
mean,35929.919772,30.000000,36.676471,239.801471,3098.983193,3107.980283,1269.154446,1275.544514,3198.305972,63535.476891,7.420546e+04,1.074693e+05,1.074546e+05,0.098930,0.030791,2020.0,1.661765,14.220588
std,20955.851366,17.606968,47.302009,201.810152,6562.560770,6560.949524,2301.359787,2300.936993,6432.870660,59783.082290,1.116034e+05,1.652658e+05,1.652421e+05,1.087044,1.071380,0.0,0.677594,8.930932
min,0.000000,0.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7881.000000,6.300000e+01,6.800000e+01,6.800000e+01,-0.772068,-2.091701,2020.0,1.000000,1.000000
25%,17533.750000,15.000000,7.000000,90.750000,101.000000,108.000000,56.000000,60.000000,110.523783,20489.750000,1.139200e+04,1.544225e+04,1.544050e+04,-0.642578,-0.842275,2020.0,1.000000,6.000000
50%,35757.500000,30.000000,18.000000,188.500000,608.000000,621.000000,308.000000,315.000000,780.459319,43536.000000,3.551950e+04,4.950050e+04,4.949250e+04,-0.332446,-0.260066,2020.0,2.000000,13.500000
75%,54027.250000,45.000000,49.000000,272.500000,2938.000000,2956.250000,1428.000000,1439.000000,3398.788264,70935.750000,8.931475e+04,1.283812e+05,1.283685e+05,0.236045,1.107154,2020.0,2.000000,22.000000
max,72335.000000,60.000000,292.000000,778.000000,67030.000000,67030.000000,21033.000000,21033.000000,78800.088474,223069.000000,1.528514e+06,2.196603e+06,2.196284e+06,3.760479,2.071858,2020.0,3.000000,31.000000


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58072 entries, 0 to 58071
Data columns (total 22 columns):
Unnamed: 0               58072 non-null int64
cityid                   58072 non-null object
takeoffdate              58072 non-null object
diff                     58072 non-null int64
cancel_num               58072 non-null int64
flight_num               58072 non-null int64
bkd_acc_c_sum            58072 non-null int64
bkd_acc_c_real_sum       58072 non-null int64
bkd_acc_q_sum            58072 non-null int64
bkd_acc_q_real_sum       58072 non-null int64
income_acc_c_real_avg    58072 non-null float64
all_seats                58072 non-null int64
direction                58072 non-null object
uv                       58072 non-null int64
pv_1                     58072 non-null int64
pv_2                     58072 non-null int64
amount                   58072 non-null float64
avg_loadfactor           58072 non-null float64
is_weekend               58072 non-null object
year   